In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [15]:
import mysql.connector

In [ ]:
mydb = mysql.connector.Connect(
    host = 'localhost'
    user = ''
    password = ''
)

بخش اول: نمودارهای تحلیلی

1) 

In [ ]:
sql1 = "Select count(book_id) category  from book_category inner join category on book_category.category_id=category.id group by category_id"
df_book1 = pd.read_sql_query(sql1, mydb)


2.

In [ ]:
sql2 = "Select count(id) publisher_name as numbook from book inner join publisher on book.publisher_id=publisher.id group by publisher_id order by numbook DES Limit 10"
df_book2 = pd.read_sql_query(sql2, mydb)

3.

In [ ]:
sql3 = "Select count(id) from book group by release_date_sh order by release_date_sh ASC"
df_book3 = pd.read_sql_query(sql3, mydb)

4.

In [ ]:
sql4 = "Select person_name count(book_id) As booknumber from book_author inner join person on book_author.person_id=person.id group by person_id order by booknumber DEC Limit 10"
df_book4 = pd.read_sql_query(sql4, mydb)

5.

In [ ]:
sql5 = "Select person_name count(book_id) As booknumber from book_translator inner join person on book_translator.person_id=person.id group by person_id order by booknumber DEC Limit 10"
df_book5 = pd.read_sql_query(sql5, mydb)

6.

In [ ]:
# Scatter Plot:
# release date sh? or gc??
sql6 = "Select release_date_sh page_count from book"
df_book6 = pd.read_sql_query(sql6, mydb)

7.

In [ ]:
# Scatter Plot:
# release year sh or gc??
sql7 = "Select release_date_sh orginal_price from book"
df_book7 = pd.read_sql_query(sql7, mydb)

8.

In [16]:
# Scatter Plot:
sql8 = "Select original_price discount_price rating from book"
df_book8 = pd.read_sql_query(sql8, mydb)

9.

In [ ]:
sql9 = "Select count(id) size_name from book inner join size on book.size_id=size.id group by size_id"
df_book9 = pd.read_sql_query(sql9, mydb)


تحلیل نمودارهای بخش اول:

بخش دوم: نمودارهای تعاملی (جستجوی پیشرفته)

In [21]:
# UI , for Filtering books
import streamlit as Strm
#

In [23]:
# Filter params:
book_name = ""
author_name = ""
has_translator = 0 # 1
book_size = 0 # num size_id
book_type_print = 0 # num type_of_print_id
book_publisher = ""
book_inventory = 1 # 0
# price :
min_price = 0
max_price = 1
# pages:
min_page_num = 0
max_page_num = 1
# print num:
min_print_num = 0
max_print_num = 1
# rating :
min_rate = 0
max_rate = 1



In [25]:
dynamic_sql = "SELECT * from "\
"(SELECT * from"\
"(SELECT * from"\
"(SELECT * from"\
"(SELECT *  from book where (discount_price>=min_price and discount_price<=max_price and rating>=min_rate and rating<=max_rate and print_series>=min_print_num and print_series<=max_print_num and page_count>=min_page_num and page_count<=max_page_num))"\
"where (size_id=book_size and type_of_print=book_type_print and inventory=book_inventory)) As table1 "\
"inner join book_author on table1.id=book_author.book_id "\
"inner join person on book_author.person_id=person.id where (author_name in person_name)) As table2 "\
"inner join publisher on table2.publisher_id=publisher.id where (book_publisher in publisher_name)) "\
"where (book_name in product_name)"

dynamic_df = pd.read_sql_query(dynamic_sql, mydb)


حالا نمایش دیتافریم ساخته‌شده برای کاربر: 

بخش سوم: تحلیل بیشتر

به صورت کلی اگر هزینه‌های مربوط به دستمزد نویسنده یا مترجم، هزینه‌های اداری، و مجوز را کنار بگذاریم، اصلی‌ترین هزینه چاپ یک کتاب مربوط به چاپ آن است. در نتیجه اگر قیمت با تخفیف یک کتاب را بر تعداد صفحه آن تقسیم کنیم، میانگین درآمد ناشر بر روی هر صفحه آن کتاب بدست می‌آید. هزینه چاپ هر صفحه به قطع آن کتاب بستگی دارد و همچنین به نوع جلد و کیفیت کاغذ بستگی دارد. در اینجا دو مورد اول را در اختیار داریم. در این بخش به صورت کلی به دنبال آن هستیم از یک سو با داشتن میزان محبوبیت کتاب که اساسی‌ترین معیار آن امتیاز دریافتی است و از سوی دیگر قیمت فروش بر هر صفحه، بهترین ژانر و نوع کتاب را با توجه به شرایط و ویژگی‌های یک ناشر مشخص کنیم. در واقع تصویری به ناشر ارائه دهیم که تغییرات قیمت کتاب نسبت به امتیاز و دیگر معیارهای محبوبیت کتاب بسته به تغییر دسته‌بندی محتوایی کتاب، نوع جلد، قطع، خارجی‌بودن یا نبودن، ناشر و تعداد تجدید چاپ؛ چگونه تغییر می‌کند.  

در اولین گام در این مسیر، سه تحلیل زیر را بر پایه داده‌های استخراج‌شده از سایت ایران کتاب ارائه می‌دهیم. تحلیل اول و دوم به دنبال بررسی ناهمگنی تغییر رفتارهای قیمتی نسبت به امتیاز در انواع قطع، جلد و دسته‌بندی است. در تحلیل سوم با الگوگیری از معیار تمرکز صنایع و سهم هر ناشر در هر یک از دسته‌بندی‌های موضوعی کتاب، میزان رقابتی‌بودن هر موضوع را بدست آورده و اثر آن بر میانگین قیمت هر صفحه کتاب در آن موضوع را مشاهده کنیم. 

تحلیل اول:

متغیر قیمت تقسیم بر تعداد صفحه با تغییر مقدار امتیاز در هر یک از انواع قطع چگونه تفاوت می‌کند؟ به عبارتی آیا تغییر نوع قطع مي‌تواند موجب تغییر روند همبستگی این دو متغیر شود؟

In [ ]:
#
sen_sql1 = "Select discount_price page_count size_id type_of_print_id rating from book inner join size on book.size_id=size.id inner join type_of_print on book.type_of_print_id=type_of_print.id where inventory=1 "
sen_df1 = pd.read_sql_query(sen_sql1, mydb)
sen_df1['priceofonepage'] = sen_df1['discount_price']/sen_df1['page_count']

# fig1:
 #priceofonepage on rating in 9 size kind!
# fig2:
 # priceofonepage on rating in 9 type_of_print !
# fig3:
# combine fig1 & fig3: 9*9 ?


تحلیل دوم:

در اینجا بررسی می‌کنیم آیا تغییرات قیمت نسبت به امتیاز در کتب ترجمه‌شده و کتب غیرترجمه‌شده یکسان است؟ ه 
از سوی دیگر دیگر در هرر بازه قیمتی، مقدار میانگین امتیاز هر یک از کتاب‌های ترجمه‌شده و نشده چگونه است؟

In [26]:
#
sen_sql2 = "Select discount_price page_count person_id Left join book_translator on book.id=book_translator.book_id where inentory=1"
sen_df2 = pd.read_sql_query(sen_sql2, mydb)
sen_df2['priceofonepage'] = sen_df2['discount_price']/sen_df2['page_count']

#sen_df2['translate']

# fig1:

# fig2:


تحلیل سوم:

در این بخش کتاب‌ها در ۱۲ دسته اصلی موضوعی، در نظر می‌گیریم و در هر موضوع، تعداد سهم هر ناشر از کل کتاب‌های آن موضوع را بدست می‌آوریم. آنگاه سهم هر ناشر را به توان ۲ رسانده و مقدار بدست آمده برای همه ناشرها را در هر موضوع جمع می کنیم. با این کار معیاری مشابه معیار سنجش میزان تمرکز در صنعت بدست می‌آید که نشانگر میزان رقابت یا انحصار در هر یک از موضوعات است. در واقع هر چه سهم یک یا چند ناشر محدود از کتاب‌های یک موضوع بیشتر باشد، مقدار این معیار بزرگتر می‌شود و هر چه رقابت بیشتر باشد، این معیار کوچک خواهد شد. از سوی دیگر یکی از عوامل موثر بر گران‌بودن یا ارزان‌بودن نسبی یک موضوع یا نوعی از کتاب می‌تواند قدرت یک ناشر یا میزان رقابت در آن موضوع باشد که میانگین قیمت بر صفحه هر موضوع را نسبت به معیار تمرکز بدست آمده، مقایسه می‌کنیم.
همچنین میانگین امتیاز هر موضوع را با معیار تمرکز مقایسه می‌کنیم.

In [ ]:
# 3
sen_sql3_cat1 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='ادبیات داستانی'"
sen_sql3_cat2 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='خودپروری'"
sen_sql3_cat3 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='روانشناسی'"
sen_sql3_cat4 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='فلسفی'"
sen_sql3_cat5 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='داستان کوتاه'"
sen_sql3_cat6 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='ادبیات نمایشی'"
sen_sql3_cat7 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='تاریخی'"
sen_sql3_cat8 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='سیاسی'"
sen_sql3_cat9 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='کتاب مصور'"
sen_sql3_cat10 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='دینی و مذهبی'"
sen_sql3_cat11 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='آموزشی'"
sen_sql3_cat12 = "Select count(id) discount_price page_count rating publisher_id category from book "\
"inner join book_category on book.id=book_category.book_id "\
  "inner joinn category on book_category.category_id=category.id group by publisher where category='کتاب کودک'"


بخش چهارم: درخواست کارفرمایان

درخواست خریدار ۱:

معیار برتری نویسنده برای کتاب‌های عاشقانه:
 میانگین امتیاز همه کتاب‌های یک نویسنده در ژانر عاشقانه ضربدر ۰.۵ 
بعلاوه
تعداد تجدید چاپ هر کتاب ضربدر امتیاز آن، جمع کل کتاب‌ها تقسیم بر کل تجدید چاپ آن نویسنده (میانگین وزن‌دار امتیازات) ضربدر ۰.۵

درخواست خریدار۲:

در اینجا کافی است، یک دیتافریم شامل کتاب‌هایی که در چارک اول بهترین‌ها از نظر امتیاز قرار دارند بسازیم و همچنین یک دیتافریم شامل کتاب‌هایی که در ۲۰ درصد ارزان‌ترین. سپس اشتراک این دو گروه را بدست آوریم.

درخواست نویسنده:

معیار بهترین ناشر برای کتاب تاریخی:
میانگین امتیاز کتب تاریخی هر ناشر ضربدر ۰.۵
 بعلاوه
میانگین وزندار امتیاز کتب تاریخی آن برحسب وزن تعداد تجدید چاپ هر کتاب ضربدر ۰.۵

تست فرض: